In [ ]:
import { OpenAI } from "npm:langchain/llms/openai";

for (let key in process.env) { // reset AZURE environment keys
  if( key.startsWith('AZURE')) {
    console.log( key )
    delete process.env[ key ] 
  }
}


const model = () => {
  try {
    return new OpenAI({
      openAIApiKey: process.env.OPENAI_API_KEY,
      azureOpenAIApiKey: undefined,
      azureOpenAIApiInstanceName: undefined,
      azureOpenAIApiDeploymentName: undefined,
      azureOpenAIApiVersion:undefined,
      // modelName: "text-davinci-003", // Defaults to "text-davinci-003" if no model provided.
      modelName: "gpt-3.5-turbo", 
      temperature: 0, 
      maxTokens: 500
    });
    
  }
  catch( e ) {
    console.log( 'ERROR', e)
  }
  
}

const llm = model()

import { PromptTemplate } from "langchain/prompts";
import { LLMChain } from "langchain/chains";

// Function to convert text to a conventional commit message
async function conventionalCommit(inputText: string): Promise<string> {
  try {
    // Use LangChain to generate a conventional commit message

    // Create a prompt with a placeholder for the commit message
    // The placeholder #~commit_message~# will be replaced with the actual commit message later
    const prompt = PromptTemplate.fromTemplate( `
    translate text provided by developer using conventional commit format following rules below
    * if text contains a file with extension, add it as scope of the commit
    * for each period add a newline into commit.
    * answer must contain only the commit text
    
    as developer: {inputText}
  `);

  const getCommitText = new LLMChain({
    llm: llm!,
    prompt,
    outputKey: "text", // For readability - otherwise the chain output will default to a property named "text"
  });

    // Generate the commit message template with the placeholder
  const result = await getCommitText.call({
    inputText: inputText, 
  });

    return result.text;
 } catch (error) {
    console.error('Error generating the commit message:', error);
    throw error
  }
}


// Run the CLI application

const messages = [
    "add note text on the vertical plane",
    'docs update readme.md',
    'use switfUI view instead of UIView. relate to issue 15',
    'package.json move to next version',
    '"10 - conventional_commit.nnb" complete conventional commit implementation '
]

const promises = messages.map( m => conventionalCommit(m))
Promise.all( promises )


